# Assessment Problems Notebook

## Problem 1 – Extending the Lady Tasting Tea

In this notebook, I will extend the Lady Tasting Tea experiment.

- Original Fisher design: 8 cups total (4 tea-first, 4 milk-first).
- Extended design: 12 cups total (8 tea-first, 4 milk-first).

A participant claims they can tell, for each cup, whether the milk or tea was poured first.  
We want to estimate, by simulation, the probability that a participant with no real ability (i.e. just guessing) correctly identifies all cups in each design.

I will:

1. Simulate the experiment many times using numpy.
2. Estimate the probability of perfectly identifying all cups by chance in:
   - The original 8-cup experiment.
   - The extended 12-cup experiment.
3. Compare the estimated probabilities and discuss what this means for p-values and evidence against the null hypothesis.